In [2]:
import gym
import random
import numpy as np
# import tensorflow as tf
#import time
from gym.envs.registration import register
from IPython.display import clear_output
from collections import deque

In [3]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() # use Tensorflow v1

2022-07-12 15:48:51.167090: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-12 15:48:51.218572: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/fernandagitai/Documents/ufal/ML/ML-Reinforcement/venv/lib/python3.8/site-packages/cv2/../../lib64:
2022-07-12 15:48:51.218585: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/fernandagitai/Documents/ufal/ML/ML-Reinforcement/venv/lib/python3.8/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the 

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
tf.compat.v1.disable_eager_execution() 

In [5]:
class Agent():
  def __init__(self, env):

    self.is_discrete = type(env.action_space) == gym.spaces.discrete.Discrete

    if self.is_discrete:
      self.action_size = env.action_space.n

    else:
      self.action_low = env.action_space.low
      self.action_high = env.action_space.high
      self.action_shape = env.action_space.shape
  
  def get_action(self, state):

    if self.is_discrete:
      #Acao randomica
      action = random.choice(range(self.action_size))
    else:
      action = np.random.uniform(self.action_low, self.action_high, self.action_shape)

    return action

In [6]:
class QNAgent(Agent):
    def __init__(self, env, discount_rate=0.97, learning_rate=0.001):
        super().__init__(env)
        self.state_size = env.observation_space.n
        print("State size:", self.state_size)
        
        self.eps = 1.0
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate
        self.build_model()

        self.sess = tf.compat.v1.Session()
        self.sess.run(tf.compat.v1.global_variables_initializer())

        self.replay_buffer = deque(maxlen=10000)
        
    def build_model(self):
        tf.compat.v1.reset_default_graph()
        self.state_in = tf.compat.v1.placeholder(tf.int32, [None])
        self.action_in = tf.compat.v1.placeholder(tf.int32, [None])
        self.target_in = tf.compat.v1.placeholder(tf.float32, [None])

        self.state = tf.one_hot(self.state_in, depth = self.state_size)
        self.action = tf.one_hot(self.action_in, depth = self.action_size)

        self.q_state = tf.compat.v1.layers.dense(self.state, units = self.action_size, name = "q_table")
        self.q_action = tf.reduce_sum(tf.multiply(self.q_state, self.action), axis = 1)

        self.loss = tf.reduce_sum(tf.square(self.target_in - self.q_action))
        self.optimizer = tf.compat.v1.train.AdamOptimizer(self.learning_rate).minimize(self.loss)
        
    def get_action(self, state):
        # q_state = self.q_table[state]
        q_state = self.sess.run(self.q_state, feed_dict={self.state_in: [state]})
        action_greedy = np.argmax(q_state)
        action_random = super().get_action(state)
        return action_random if random.random() < self.eps else action_greedy
    
    def train(self, experience, batch_size=50):
        self.replay_buffer.append(experience)
        samples = random.choices(self.replay_buffer, k=batch_size)
        state, action, next_state, reward, done = (list(col) for col in zip(experience, *samples))
        
        q_next = self.sess.run(self.q_state, feed_dict={self.state_in: next_state})
        q_next[done] = np.zeros([self.action_size])
        q_target = reward + self.discount_rate * np.max(q_next, axis =1)

        feed = {self.state_in: state, self.action_in: action, self.target_in: q_target}
        self.sess.run(self.optimizer, feed_dict=feed)
        
        if experience[4]:
            self.eps = self.eps * 0.99

    def __del__(self):
        self.sess.close()

In [12]:
# env = gym.make('FrozenLake-v1', is_slippery=True, map_name='4x4')
env = gym.make('FrozenLake-v1', is_slippery=True, map_name='8x8')
agent = QNAgent(env)
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)

State size: 64
Observation space: Discrete(64)
Action space: Discrete(4)


/tmp/ipykernel_16880/640231698.py:26: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  self.q_state = tf.compat.v1.layers.dense(self.state, units = self.action_size, name = "q_table")


In [14]:
total_reward = 0
for ep in range(300):
    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        agent.train((state,action,next_state,reward,done))
        state = next_state
        total_reward += reward
        print("s:", state, "a:", action)
        print("Episode: {}, Total reward: {}, eps: {}".format(ep,total_reward,agent.eps))
        env.render()

        with tf.compat.v1.variable_scope("q_table", reuse = True):
            weights = agent.sess.run(tf.compat.v1.get_variable("kernel"))
            print(weights)
            # scope.reuse_variables()
        #print(agent.q_table)
        clear_output(wait=True)

s: 46 a: 2
Episode: 299, Total reward: 102.0, eps: 0.002405009291311067
[[ 1.54020060e-02 -1.39644491e-02 -4.04188298e-02 -1.01721743e-02]
 [ 1.47264246e-02 -1.10188164e-02 -3.58215272e-02 -8.29403847e-03]
 [ 2.51656920e-02 -5.75790182e-03 -3.28460336e-02 -4.85578924e-03]
 [-8.40914086e-04  3.64607922e-03 -1.87608823e-02  2.10211403e-03]
 [ 3.27999182e-02  1.95628908e-02 -3.41291539e-03  1.65083408e-02]
 [ 6.52983263e-02  2.96030957e-02  1.08784623e-02  3.47621106e-02]
 [ 7.21552670e-02  4.21273932e-02  2.03960463e-02  2.08594017e-02]
 [ 7.86524415e-02  5.11244014e-02  3.23383100e-02  4.83799353e-02]
 [ 7.04795728e-03 -2.16180161e-02 -4.43882644e-02 -1.38280177e-02]
 [ 4.28885315e-03 -2.28600968e-02 -4.04414013e-02 -1.00907451e-02]
 [-3.34464200e-02 -3.84249389e-02 -4.46309410e-02 -6.03523990e-03]
 [-6.99154809e-02 -1.17733166e-01 -5.52530736e-02  3.05490772e-04]
 [ 3.87766100e-02  1.34623526e-02 -5.53117692e-03  1.97186489e-02]
 [ 4.65569422e-02  3.69274914e-02  1.44754024e-02  4.0895

In [15]:
env.close()